In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Prueba de hipótesis

Para determinar qué prueba de hipótesis realizar con este conjunto de datos, primero debemos identificar la pregunta o la afirmación que deseamos investigar. Aquí planteamos algunos ejemplos de preguntas o afirmaciones que probaremos para entender en que consiste la prueba de hipótesis:

1. **Hipótesis sobre los ingresos promedio en función de la educación:**
   - Hipótesis nula (H0): No hay diferencia en los ingresos promedio entre las personas con educación básica 4 años y las personas con educación secundaria.
   - Hipótesis alternativa (H1): Hay una diferencia significativa en los ingresos promedio entre estos dos grupos.
   - Prueba de hipótesis: Se podría realizar una prueba t de Student para comparar los ingresos promedio de estos dos grupos.

2. **Hipótesis sobre la duración de la llamada telefónica en función del estado civil:**
   - Hipótesis nula (H0): No hay diferencia en la duración promedio de las llamadas telefónicas entre personas casadas y personas solteras.
   - Hipótesis alternativa (H1): Hay una diferencia significativa en la duración promedio de las llamadas telefónicas entre estos dos grupos.
   - Prueba de hipótesis: También se podría utilizar una prueba t de Student para comparar la duración promedio de las llamadas entre los dos grupos de estado civil.

Para poder enfrentarnos a estos problemas vamos a tener que usar el método `ttest_ind()` de la librería `stats`.  Lo que hace este método es usar el "Test de Student".

La función `ttest_ind` se utiliza para realizar una prueba t de Student independiente entre dos muestras (grupos) para comparar si sus medias son estadísticamente diferentes. Aquí está la sintaxis básica:

```python
ttest_ind(a, b, axis, equal_var, nan_policy=)
```


Uno de los parámetros que tenemos en el `ttest_ind` es el `equal_var`, donde le tenemos que decir, si nuestras categorías a comparar tienen varianzas iguales o no. Por lo tanto, lo primero que tendremos que hacer es evaluarlo, para eso tenemos dos test: **Prueba de Levene**
y la **Prueba de Bartlett**. 

> En ambos casos, si el p-valor < 0.05 podemos concluir que las varianzas son diferentes entre los grupos. Si el p-valor > 0.05, no podemos afirmar que las varianzas son diferentes.

In [2]:
# lo primero que hacemos es cargar el dataframe que vamos a usar para esta lección
df = pd.read_csv("bank-additional_clean.csv", index_col = 0)
df.head(2)

,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,y,date,latitude,longitude,contact_month,contact_year,age_cat,hijos_totales
0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,NaN,housemaid,married,basic 4y,No,No,No,telephone,261,1,NaN,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no,2-agosto-2019,41.495,-71.233,agosto,2019.0,Adultos mayores,1 hijos
1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,57.0,services,married,high school,NaN,No,No,telephone,149,1,NaN,0,nonexistent,1.1,93.994,-36.4,NaN,5191,no,14-septiembre-2016,34.601,-83.923,septiembre,2016.0,Mediana edad,2 hijos


In [3]:
# vamos a definir una función para poder hacer el t-test de Student

def prueba_hipotesis(*args):
    
    # lo primero que tenemos que hacer es mirar si las varianzas son iguales o no
    if len(args) == 2:
        p_valor_varianza = stats.levene(*args, center = "median")[1]
    else:
        p_valor_varianza = stats.bartlett(*args)[1]
    
    if p_valor_varianza > 0.05:
        # realizamos la prueba t de Student
        t_stat, p_valor = stats.ttest_ind(*args, equal_var=True)
    else:
        t_stat, p_valor = stats.ttest_ind(*args, equal_var=False)
        
    # Establecemos un nivel de significancia (alfa)
    alfa = 0.05

    # comparamos el p-valor con el nivel de significancia
    if p_valor < alfa:
        print("Rechazamos la hipótesis nula.")
        print("Hay una diferencia significativa en los ingresos promedio entre los dos grupos.")
    else:
        print("No podemos rechazar la hipótesis nula.")
        print("No hay evidencia suficiente para afirmar una diferencia significativa en los ingresos promedio entre los dos grupos.")

In [4]:

# filtramos los datos para obtener dos grupos: educación básica 4 años y educación secundaria
grupo_educacion_basica = df[df['education'] == 'basic 4y']["income"]
grupo_educacion_secundaria = df[df['education'] == 'high school']["income"]

# llamamos a la función que hemos creado
prueba_hipotesis(grupo_educacion_basica, grupo_educacion_secundaria )


Rechazamos la hipótesis nula.
Hay una diferencia significativa en los ingresos promedio entre los dos grupos.


In [5]:
# vamos a probar la segunda hipótesis que tenemos planteada
# No hay diferencia en la duración promedio de las llamadas telefónicas entre personas casadas y personas solteras.

# lo primero que tenemos que hacer es crear dos DataFrames nuevos, uno para saber el tiempo de las llamadas a personas casadas y otro para personas solteras
grupo_casadas = df[df["marital"] == "married"]["duration"]
grupo_solteras = df[df["marital"] == "single"]["duration"]

# llamamos a la función que hemos creado
prueba_hipotesis(grupo_casadas, grupo_solteras )

No podemos rechazar la hipótesis nula.
No hay evidencia suficiente para afirmar una diferencia significativa en los ingresos promedio entre los dos grupos.


# Ejercicios

Seguiremos usando el conjunto de datos que usamos siempre para los ejercicios de la clase invertida. 

1. Dadas las situaciones, decide si se trata de una distribución de tipo exponencial o de poisson. A nivel teórico, no hace falta que lo demuestres con código. 

   - Imagina que estás registrando el tiempo que transcurre entre la llegada de clientes a una tienda en minutos. ¿Se asemeja más a una distribución exponencial o de Poisson? EXPONENCIAL

   - Si observas el número de accidentes que ocurren en una intersección en un día determinado, ¿tiene más sentido pensar en una distribución exponencial o de Poisson? POISSON

   - Cuando las personas esperan en una cola para recibir atención al cliente en una tienda, se registra el tiempo que esperan en minutos. ¿Parece seguir una distribución exponencial o de Poisson? EXPONENCIAL

   - Si supervisas el número de llamadas telefónicas que llegan a un centro de atención al cliente en una hora, ¿sería más apropiado pensar en una distribución exponencial o de Poisson? POISSON

   - Si mides el tiempo entre la llegada de correos electrónicos a tu bandeja de entrada durante el día, ¿se ajusta mejor a una distribución exponencial o de Poisson? EXPONENCIAL

In [6]:
df = pd.read_csv('df_nulos_imputados.csv', index_col=0)

2. Ejercicios intervalo de confianza: Para cada uno de los ejercicios interpreta los resultados y piensa que ventaja a nivel de negocio podemos sacar de calcularlo. 

   - Trabajas para una plataforma de comercio en línea y deseas determinar el intervalo de confianza al 95% para el precio promedio de productos con una cantidad de media de valoraciones al producto de 500.


In [23]:
# Filtrar productos con una media de valoraciones cercana a 500 (por ejemplo, entre 450 y 550) >> Esa es la muestra 
filtered_df = df[(df['mean_product_ratings_count'] >= 450) & (df['mean_product_ratings_count'] <= 550)]

# Calcular la media y la desviación estándar de los precios de los productos filtrados
mean_price = filtered_df['mean_product_prices'].mean()
std_dev_price = filtered_df['mean_product_prices'].std()
n = filtered_df.shape[0]  # Número de productos en el filtro

# Calcular el intervalo de confianza al 95%
confidence_level = 0.95
z_value = stats.norm.ppf(0.5 + confidence_level / 2)  # Z value for 95% confidence
margin_of_error = z_value * (std_dev_price / np.sqrt(n))

# Calcular el intervalo de confianza
lower_bound = mean_price - margin_of_error
upper_bound = mean_price + margin_of_error

print(f"El intervalo de confianza al 95% para el precio promedio de productos con una cantidad de media de valoraciones al producto de 500 es: ({lower_bound}, {upper_bound})")


El intervalo de confianza al 95% para el precio promedio de productos con una cantidad de media de valoraciones al producto de 500 es: (7.97593555702236, 10.068903152655057)


In [24]:
# Filtrar productos con una media de valoraciones cercana a 500 (por ejemplo, entre 450 y 550) >> Esa es la muestra 
filtered_df = df[(df['mean_product_ratings_count'] >= 450) & (df['mean_product_ratings_count'] <= 550)]

# Calcular la media y el error estándar de la muestra
media_precio = filtered_df["mean_product_prices"].mean()
error_precio = stats.sem(filtered_df["mean_product_prices"])

# Definir el nivel de confianza (95% en este caso)
nivel_confianza_precio = 0.95

# calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1. 
grados_libertad_precio = len(filtered_df["mean_product_prices"]) - 1

# Calcular el valor crítico (utilizando la distribución t de Student)
valor_critico_precio = stats.t.ppf((1 + nivel_confianza_precio) / 2, df=grados_libertad_precio)

# Calcular el intervalo de confianza
limite_inferior_precio = media_precio - valor_critico_precio * error_precio
limite_superior_precio = media_precio + valor_critico_precio * error_precio

print("Intervalo de Confianza para el precio promedio de productos con una cantidad de media de valoraciones al producto de 500:")
print(f"Media Muestral: {np.round(media_precio, 2)}")
print(f"Error Estándar: {np.round(error_precio, 2)}")
print(f"Nivel de Confianza: {nivel_confianza_precio}")
print(f"Valor Crítico: {np.round(valor_critico_precio, 2)}")
print(f"Intervalo de Confianza: ({np.round(limite_inferior_precio, 2)}, {np.round(limite_superior_precio, 2)})")

Intervalo de Confianza para el precio promedio de productos con una cantidad de media de valoraciones al producto de 500:
Media Muestral: 9.02
Error Estándar: 0.53
Nivel de Confianza: 0.95
Valor Crítico: 2.0
Intervalo de Confianza: (7.95, 10.09)



   - Eres un analista de ventas en una plataforma de comercio electrónico y quieres calcular el intervalo de confianza al 90% para el número promedio de unidades vendidas por producto.

In [27]:
# Calcular la media y el error estándar de la muestra
media_unidades_vendidas = df["mean_units_sold_per_product"].mean()
error_unidades_vendidas = stats.sem(df["mean_units_sold_per_product"])

# Definir el nivel de confianza (90% en este caso)
nivel_confianza_unidades_vendidas = 0.90

# calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1. 
grados_libertad_unidades_vendidas = len(df["mean_units_sold_per_product"]) - 1

# Calcular el valor crítico (utilizando la distribución t de Student)
valor_critico_unidades_vendidas = stats.t.ppf((1 + nivel_confianza_unidades_vendidas) / 2, df=grados_libertad_unidades_vendidas)

# Calcular el intervalo de confianza
limite_inferior_unidades_vendidas = media_unidades_vendidas - valor_critico_unidades_vendidas * error_unidades_vendidas
limite_superior_unidades_vendidas = media_unidades_vendidas + valor_critico_unidades_vendidas * error_unidades_vendidas

print("Intervalo de Confianza para el número promedio de unidades vendidas por producto:")
print(f"Media Muestral: {np.round(media_unidades_vendidas, 2)}")
print(f"Error Estándar: {np.round(error_unidades_vendidas, 2)}")
print(f"Nivel de Confianza: {nivel_confianza_unidades_vendidas}")
print(f"Valor Crítico: {np.round(valor_critico_unidades_vendidas, 2)}")
print(f"Intervalo de Confianza: ({np.round(limite_inferior_unidades_vendidas, 2)}, {np.round(limite_superior_unidades_vendidas, 2)})")

Intervalo de Confianza para el número promedio de unidades vendidas por producto:
Media Muestral: 4339.01
Error Estándar: 202.91
Nivel de Confianza: 0.9
Valor Crítico: 1.65
Intervalo de Confianza: (4005.05, 4672.97)


   - Trabajas en la gestión de inventario de una tienda en línea y necesitas calcular el intervalo de confianza al 99% para el precio promedio de productos según su país de origen.

In [38]:
df.origin_country.unique()

array(['CN', 'US', 'VE', 'AT', 'SG', 'GB'], dtype=object)

In [54]:
# Crear un diccionario para almacenar los DataFrames de cada país
df_dict = {}

# Iterar sobre los países únicos en la columna 'origin_country'
for country in df.origin_country.unique():
    # Crear un DataFrame para cada país
    df_dict[country] = df[df['origin_country'] == country]

# Definir el nivel de confianza (99% en este caso)
nivel_confianza = 0.99

# Calcular el intervalo de confianza para cada grupo
for k, v in df_dict.items():
    # Verificar la cantidad de observaciones en el grupo
    if len(v['mean_product_prices']) < 2:
        print(f"Intervalo de Confianza para el precio promedio de productos según su país de origen: {k}")
        print("No se puede calcular el intervalo de confianza debido a la cantidad insuficiente de observaciones.")
        print('---'*30)
        continue
    
    # Calcular la media y el error estándar de la muestra
    media = v['mean_product_prices'].mean()
    error = stats.sem(v["mean_product_prices"])

    # Calcular los grados de libertad de la muestra
    grados_libertad = len(v["mean_product_prices"]) - 1

    # Calcular el valor crítico (utilizando la distribución t de Student)
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)

    # Calcular el intervalo de confianza
    limite_inferior = media - valor_critico * error
    limite_superior = media + valor_critico * error

    print(f"Intervalo de Confianza para el precio promedio de productos según su país de origen: {k}")
    print(f"Media Muestral: {np.round(media, 2)}")
    print(f"Error Estándar: {np.round(error, 2)}")
    print(f"Nivel de Confianza: {nivel_confianza}")
    print(f"Valor Crítico: {np.round(valor_critico, 2)}")
    print(f"Intervalo de Confianza: ({np.round(limite_inferior, 2)}, {np.round(limite_superior, 2)})")
    print('---'*30)

Intervalo de Confianza para el precio promedio de productos según su país de origen: CN
Media Muestral: 8.31
Error Estándar: 0.09
Nivel de Confianza: 0.99
Valor Crítico: 2.58
Intervalo de Confianza: (8.07, 8.55)
------------------------------------------------------------------------------------------
Intervalo de Confianza para el precio promedio de productos según su país de origen: US
Media Muestral: 9.4
Error Estándar: 0.5
Nivel de Confianza: 0.99
Valor Crítico: 2.75
Intervalo de Confianza: (8.03, 10.76)
------------------------------------------------------------------------------------------
Intervalo de Confianza para el precio promedio de productos según su país de origen: VE
Media Muestral: 8.3
Error Estándar: 2.27
Nivel de Confianza: 0.99
Valor Crítico: 4.6
Intervalo de Confianza: (-2.14, 18.73)
------------------------------------------------------------------------------------------
Intervalo de Confianza para el precio promedio de productos según su país de origen: AT
No s

   - Eres un gerente de ventas en una plataforma de comercio electrónico y quieres determinar el intervalo de confianza al 95% para la valoración promedio de los vendedores según si ofrecen envío express o no.


In [ ]:
# Valoración promedio de los vendedores, no sé qué columna es esa??

   - Trabajas en el departamento de precios de una tienda en línea y necesitas calcular el intervalo de confianza al 90% para el descuento promedio de productos según la reputación del vendedor (alta, media o baja).

In [57]:
# Crear un diccionario para almacenar los DataFrames de cada país
df_dict = {}

# Iterar sobre los países únicos en la columna 'origin_country'
for rep in df.seller_reputation.unique():
    # Crear un DataFrame para cada país
    df_dict[rep] = df[df['seller_reputation'] == rep]

# Definir el nivel de confianza (90% en este caso)
nivel_confianza = 0.90

# Calcular el intervalo de confianza para cada grupo
for k, v in df_dict.items():
    # Verificar la cantidad de observaciones en el grupo
    if len(v['discount_percentage']) < 2:
        print(f"Intervalo de Confianza para el descuento promedio de productos según la reputación del vendedor: {k}")
        print("No se puede calcular el intervalo de confianza debido a la cantidad insuficiente de observaciones.")
        print('---'*30)
        continue
    
    # Calcular la media y el error estándar de la muestra
    media = v['discount_percentage'].mean()
    error = stats.sem(v["discount_percentage"])

    # Calcular los grados de libertad de la muestra
    grados_libertad = len(v["discount_percentage"]) - 1

    # Calcular el valor crítico (utilizando la distribución t de Student)
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)

    # Calcular el intervalo de confianza
    limite_inferior = media - valor_critico * error
    limite_superior = media + valor_critico * error

    print(f"Intervalo de Confianza para ara el descuento promedio de productos según la reputación del vendedor: {k}")
    print(f"Media Muestral: {np.round(media, 2)}")
    print(f"Error Estándar: {np.round(error, 2)}")
    print(f"Nivel de Confianza: {nivel_confianza}")
    print(f"Valor Crítico: {np.round(valor_critico, 2)}")
    print(f"Intervalo de Confianza: ({np.round(limite_inferior, 2)}, {np.round(limite_superior, 2)})")
    print('---'*30)

Intervalo de Confianza para ara el descuento promedio de productos según la reputación del vendedor: Buena
Media Muestral: 26.43
Error Estándar: 1.34
Nivel de Confianza: 0.9
Valor Crítico: 1.65
Intervalo de Confianza: (24.22, 28.64)
------------------------------------------------------------------------------------------
Intervalo de Confianza para ara el descuento promedio de productos según la reputación del vendedor: Regular
Media Muestral: 25.53
Error Estándar: 1.61
Nivel de Confianza: 0.9
Valor Crítico: 1.65
Intervalo de Confianza: (22.87, 28.18)
------------------------------------------------------------------------------------------


3. Ejercicios prueba de hipótesis:

   - Prueba de Hipótesis sobre el precio promedio de productos con o sin empleados calificados

      - *Contexto:* Trabajas en una plataforma de comercio en línea que ofrece productos de moda. Quieres saber si hay una diferencia significativa en el precio promedio de productos entre aquellos vendidos por empleados calificados (rating_employee > 4) y aquellos vendidos por empleados menos calificados (rating_employee <= 4). Esto te ayudará a tomar decisiones sobre la estrategia de precios y capacitación de empleados.

      - *Hipótesis Nula (H0):* No hay diferencia significativa en el precio promedio de productos entre empleados con buena y mala reputación. 
      
      - *Hipótesis Alternativa (H1):* Existe una diferencia significativa en el precio promedio de productos entre empleados calificados y menos calificados.

   - Prueba de Hipótesis sobre el número de unidades vendidas en envío express y envío estándar

      - *Contexto:* Eres un gerente de operaciones de envío en una plataforma de comercio electrónico. Quieres determinar si el número promedio de unidades vendidas difiere cuando los productos se envían con envío express (shipping_option_name = "Express Shipping") en comparación con envío estándar (shipping_option_name = "Livraison standard"). Esto te ayudará a tomar decisiones sobre la logística de envío.

      - *Hipótesis Nula (H0):* No hay diferencia significativa en el número promedio de unidades vendidas entre envío express y envío estándar.

      - *Hipótesis Alternativa (H1):* Existe una diferencia significativa en el número promedio de unidades vendidas entre envío express y envío estándar.
